# Assignment 2: Hybrid Images and Scale-space blob detection

In [1]:
# Libraries you will find useful
import numpy as np
import scipy 
import skimage
import matplotlib.pyplot as plt

## Part 1: Hybrid Images

In [13]:
# Crop and align the images such that the objects and their edges are aligned. 
# You are free to use any image editing tool for this and there is no need for code for this step.

# Load cropped and aligned images

In [12]:
# Read aligned input images and convert them to grayscale 

In [11]:
# Apply a low-pass filter, i.e., a standard 2D Gaussian filter, on the first (smooth) image. 
from scipy.ndimage import gaussian_filter

In [ ]:
# Apply a high-pass filter on the second image. 
# The paper suggests using an impulse (identity) filter minus a Gaussian filter for this operation.

In [ ]:
# Use your intuition and trial and error to determine good values of σ for the high-pass and low-pass filters
# One of the σ's should always be higher than the other (which one?), but the optimal values can vary from image to image.

In [ ]:
# Add or average the tranformed images to create the hybrid image.

## Part 2: Scale-space blob detection

In [ ]:
# convert images to grayscale
# rescale the intensities to between 0 and 1 (simply divide them by 255 should do the trick)

In [10]:
# Creating the Laplacian filter
# Pay careful attention to setting the right filter mask size. Hint: Should the filter width be odd or even?
from scipy.ndimage.filters import gaussian_laplace

In [7]:
# filtering the image (two implmementations)
# one that increases filter size, and one that downsamples the image
# For timing, use time.time()

In [ ]:
# nonmaximum suppression in scale space
# you may find functions scipy.ndimage.filters.rank_filter or scipy.ndimage.filters.generic_filter useful

In [ ]:
# To display the detected regions as circle
from matplotlib.patches import Circle
def show_all_circles(image, cx, cy, rad, color='r'):
    """
    image: numpy array, representing the grayscsale image
    cx, cy: numpy arrays or lists, centers of the detected blobs
    rad: numpy array or list, radius of the detected blobs
    """
    fig, ax = plt.subplots()
    ax.set_aspect('equal')
    ax.imshow(image, cmap='gray')
    for x, y, r in zip(cx, cy, rad):
        circ = Circle((x, y), r, color=color, fill=False)
        ax.add_patch(circ)

    plt.title('%i circles' % len(cx))
    plt.show()